In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import shutil
import sys
import numpy as np
import pandas as pd
import tensorflow as tf

In [65]:
_CSV_COLUMNS = [
    '', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 
    'target', 'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
]

_CSV_COLUMN_DEFAULTS = [[0], [''], [''], [''], [''], [''], 
                        [''], [0.0], [''], [''], [''], [''], 
                        [0.0], [0], [0], [''], [0], [0], [0]]

parser = argparse.ArgumentParser()

parser.add_argument(
    '--model_dir', type=str, default='/tmp/census_model',
    help='Base directory for the model.')

parser.add_argument(
    '--model_type', type=str, default='wide_deep',
    help="Valid model types: {'wide', 'deep', 'wide_deep'}.")

parser.add_argument(
    '--train_epochs', type=int, default=40, help='Number of training epochs.')

parser.add_argument(
    '--epochs_per_eval', type=int, default=2,
    help='The number of training epochs to run between evaluations.')

parser.add_argument(
    '--batch_size', type=int, default=40, help='Number of examples per batch.')

parser.add_argument(
    '--train_data', type=str, default='./labeled_train_X2.csv',
    help='Path to the training data.')

parser.add_argument(
    '--test_data', type=str, default='./labeled_test_X2.csv',
    help='Path to the test data.')

_StoreAction(option_strings=['--test_data'], dest='test_data', nargs=None, const=None, default='./labeled_test_X2.csv', type=<class 'str'>, choices=None, help='Path to the test data.', metavar=None)

In [60]:
_NUM_EXAMPLES = {
    'train': 5901935,
    'validation': 1475484,
}


In [66]:
def build_model_columns():
  """Builds a set of wide and deep feature columns."""
  # Continuous columns
  bd = tf.feature_column.numeric_column('bd')
  song_length = tf.feature_column.numeric_column('song_length')
  city = tf.feature_column.numeric_column('city')
  language = tf.feature_column.numeric_column('language')
  registered_via = tf.feature_column.numeric_column('registered_via')
  
  '''language = tf.feature_column.categorical_column_with_hash_bucket(
      'language', hash_bucket_size=300)
  city = tf.feature_column.categorical_column_with_hash_bucket(
      'city', hash_bucket_size=2000)'''

  gender = tf.feature_column.categorical_column_with_vocabulary_list(
      'gender', ['male', 'female'])

  #msno = tf.feature_column.categorical_column_with_hash_bucket(
  #    'msno', hash_bucket_size=6000)
  artist_name = tf.feature_column.categorical_column_with_hash_bucket(
      'artist_name', hash_bucket_size=2000)
  composer = tf.feature_column.categorical_column_with_hash_bucket(
      'composer', hash_bucket_size=2000)
  lyricist = tf.feature_column.categorical_column_with_hash_bucket(
      'lyricist', hash_bucket_size=2000)
  # To show an example of hashing:
  source_system_tab = tf.feature_column.categorical_column_with_hash_bucket(
      'source_system_tab', hash_bucket_size=10)
  source_screen_name = tf.feature_column.categorical_column_with_hash_bucket(
      'source_screen_name', hash_bucket_size=10)
  source_type = tf.feature_column.categorical_column_with_hash_bucket(
      'source_type', hash_bucket_size=10)

  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(
      bd, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

  # Wide columns and deep columns.
  base_columns = [
      gender, artist_name, source_system_tab, source_screen_name,
      source_type, composer, lyricist, age_buckets,#msno
  ]

  crossed_columns = [
      tf.feature_column.crossed_column(
          ['artist_name', 'composer', 'lyricist'], hash_bucket_size=10000),
  ]

  wide_columns = base_columns + crossed_columns

  deep_columns = [
      bd,
      song_length,
      city,
      language,
      registered_via,
  ]

  return wide_columns, deep_columns

In [67]:
def build_estimator(model_dir, model_type):
  """Build an estimator appropriate for the given model type."""
  wide_columns, deep_columns = build_model_columns()
  hidden_units = [100, 75, 50, 25]

  # Create a tf.estimator.RunConfig to ensure the model is run on CPU, which
  # trains faster than GPU for this model.
  run_config = tf.estimator.RunConfig().replace(
      session_config=tf.ConfigProto(device_count={'GPU': 0}))

  if model_type == 'wide':
    return tf.estimator.LinearClassifier(
        model_dir=model_dir,
        feature_columns=wide_columns,
        config=run_config)
  elif model_type == 'deep':
    return tf.estimator.DNNClassifier(
        model_dir=model_dir,
        feature_columns=deep_columns,
        hidden_units=hidden_units,
        config=run_config)
  else:
    return tf.estimator.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units,
        config=run_config)

In [68]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, field_delim = ',', record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('target')
    return features, tf.equal(labels, '1')

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)
  
  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5) #skip(7376419)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)

  iterator = dataset.make_one_shot_iterator()
  features, labels = iterator.get_next()
  return features, labels

In [69]:
temp = 0
temp2 = 0
def main(unused_argv):
  global temp
  global temp2
  # Clean up the model directory if present
  shutil.rmtree(FLAGS.model_dir, ignore_errors=True)
  model = build_estimator(FLAGS.model_dir, FLAGS.model_type)

  # Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
  for n in range(1):#range(FLAGS.train_epochs // FLAGS.epochs_per_eval):
    model.train(input_fn=lambda: input_fn(
        FLAGS.train_data, FLAGS.epochs_per_eval, True, FLAGS.batch_size))

    #results = model.evaluate(input_fn=lambda: input_fn(
    #    FLAGS.test_data, 1, False, FLAGS.batch_size))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * FLAGS.epochs_per_eval)
    print('-' * 60)

    #for key in sorted(results):
    #  print('%s: %s' % (key, results[key]))
  #model.export_savedmodel(export_dir_base='./')
  column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
  df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                            skip_blank_lines=True, keep_default_na=False)#, dtype={'city':str, 'language':str})
  predictions = list(model.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
      x=df, num_epochs=1,shuffle=False)))
  res = pd.DataFrame(predictions)
  temp = model
  temp2 = predictions
  #print(res.count)
  res.to_csv('pred.csv', sep = ',', header = False)
  

if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.INFO)
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed + [temp])

INFO:tensorflow:Using config: {'_tf_random_seed': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_session_config': device_count {
  key: "GPU"
}
, '_task_type': 'worker', '_model_dir': '/tmp/census_model', '_task_id': 0, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_service': None, '_keep_checkpoint_max': 5, '_master': '', '_is_chief': True, '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a0751978>, '_save_checkpoints_steps': None}
Parsing ./labeled_train_X2.csv
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/census_model/model.ckpt.
INFO:tensorflow:loss = 164082.0, step = 1
INFO:tensorflow:global_step/sec: 60.6072
INFO:tensorflow:loss = 14350.5, step = 101 (1.655 sec)
INFO:tensorflow:global_step/sec: 96.6645
INFO:tensorflow:loss = 966.539, step = 201 (1.037 sec)
INFO:tensorflow:global_step/sec: 83.4134
INFO:tens

INFO:tensorflow:global_step/sec: 124.9
INFO:tensorflow:loss = 657.302, step = 7801 (0.796 sec)
INFO:tensorflow:global_step/sec: 103.814
INFO:tensorflow:loss = 606.082, step = 7901 (0.964 sec)
INFO:tensorflow:global_step/sec: 87.9888
INFO:tensorflow:loss = 355.443, step = 8001 (1.137 sec)
INFO:tensorflow:global_step/sec: 169.421
INFO:tensorflow:loss = 396.85, step = 8101 (0.589 sec)
INFO:tensorflow:global_step/sec: 146.24
INFO:tensorflow:loss = 776.436, step = 8201 (0.686 sec)
INFO:tensorflow:global_step/sec: 156.242
INFO:tensorflow:loss = 1403.47, step = 8301 (0.638 sec)
INFO:tensorflow:global_step/sec: 156.48
INFO:tensorflow:loss = 313.699, step = 8401 (0.640 sec)
INFO:tensorflow:global_step/sec: 167.217
INFO:tensorflow:loss = 92.8309, step = 8501 (0.598 sec)
INFO:tensorflow:global_step/sec: 165.942
INFO:tensorflow:loss = 730.963, step = 8601 (0.605 sec)
INFO:tensorflow:global_step/sec: 65
INFO:tensorflow:loss = 419.663, step = 8701 (1.546 sec)
INFO:tensorflow:global_step/sec: 116.106

INFO:tensorflow:loss = 664.18, step = 16201 (0.720 sec)
INFO:tensorflow:global_step/sec: 150.218
INFO:tensorflow:loss = 426.638, step = 16301 (0.673 sec)
INFO:tensorflow:global_step/sec: 135.71
INFO:tensorflow:loss = 589.076, step = 16401 (0.730 sec)
INFO:tensorflow:global_step/sec: 168.386
INFO:tensorflow:loss = 526.546, step = 16501 (0.594 sec)
INFO:tensorflow:global_step/sec: 169.796
INFO:tensorflow:loss = 918.432, step = 16601 (0.588 sec)
INFO:tensorflow:global_step/sec: 169.046
INFO:tensorflow:loss = 69.8722, step = 16701 (0.593 sec)
INFO:tensorflow:global_step/sec: 185.897
INFO:tensorflow:loss = 708.025, step = 16801 (0.539 sec)
INFO:tensorflow:global_step/sec: 174.996
INFO:tensorflow:loss = 47.6874, step = 16901 (0.568 sec)
INFO:tensorflow:global_step/sec: 154.929
INFO:tensorflow:loss = 281.315, step = 17001 (0.646 sec)
INFO:tensorflow:global_step/sec: 156.194
INFO:tensorflow:loss = 165.863, step = 17101 (0.645 sec)
INFO:tensorflow:global_step/sec: 151.189
INFO:tensorflow:loss =

INFO:tensorflow:loss = 782.147, step = 24601 (0.689 sec)
INFO:tensorflow:global_step/sec: 208.6
INFO:tensorflow:loss = 456.027, step = 24701 (0.481 sec)
INFO:tensorflow:global_step/sec: 122.49
INFO:tensorflow:loss = 189.388, step = 24801 (0.816 sec)
INFO:tensorflow:global_step/sec: 154.489
INFO:tensorflow:loss = 50.9634, step = 24901 (0.647 sec)
INFO:tensorflow:global_step/sec: 167.235
INFO:tensorflow:loss = 24.7536, step = 25001 (0.598 sec)
INFO:tensorflow:global_step/sec: 207.18
INFO:tensorflow:loss = 493.575, step = 25101 (0.483 sec)
INFO:tensorflow:global_step/sec: 153.8
INFO:tensorflow:loss = 326.469, step = 25201 (0.650 sec)
INFO:tensorflow:global_step/sec: 232.441
INFO:tensorflow:loss = 206.347, step = 25301 (0.430 sec)
INFO:tensorflow:global_step/sec: 145.493
INFO:tensorflow:loss = 817.25, step = 25401 (0.687 sec)
INFO:tensorflow:global_step/sec: 237.438
INFO:tensorflow:loss = 303.374, step = 25501 (0.421 sec)
INFO:tensorflow:global_step/sec: 130.18
INFO:tensorflow:loss = 661.5

INFO:tensorflow:loss = 647.72, step = 33001 (0.688 sec)
INFO:tensorflow:global_step/sec: 152.862
INFO:tensorflow:loss = 105.776, step = 33101 (0.654 sec)
INFO:tensorflow:global_step/sec: 138.068
INFO:tensorflow:loss = 298.861, step = 33201 (0.725 sec)
INFO:tensorflow:global_step/sec: 235.068
INFO:tensorflow:loss = 291.571, step = 33301 (0.425 sec)
INFO:tensorflow:global_step/sec: 142.42
INFO:tensorflow:loss = 294.016, step = 33401 (0.702 sec)
INFO:tensorflow:global_step/sec: 158.674
INFO:tensorflow:loss = 27.647, step = 33501 (0.631 sec)
INFO:tensorflow:global_step/sec: 153.361
INFO:tensorflow:loss = 156.095, step = 33601 (0.651 sec)
INFO:tensorflow:global_step/sec: 185.744
INFO:tensorflow:loss = 174.709, step = 33701 (0.538 sec)
INFO:tensorflow:global_step/sec: 198.629
INFO:tensorflow:loss = 39.2433, step = 33801 (0.503 sec)
INFO:tensorflow:global_step/sec: 141.504
INFO:tensorflow:loss = 194.939, step = 33901 (0.707 sec)
INFO:tensorflow:global_step/sec: 111.042
INFO:tensorflow:loss = 

...

INFO:tensorflow:global_step/sec: 244.34
INFO:tensorflow:loss = 101.68, step = 349301 (0.409 sec)
INFO:tensorflow:global_step/sec: 243.347
INFO:tensorflow:loss = 136.378, step = 349401 (0.411 sec)
INFO:tensorflow:global_step/sec: 145.528
INFO:tensorflow:loss = 75.6605, step = 349501 (0.689 sec)
INFO:tensorflow:global_step/sec: 125.421
INFO:tensorflow:loss = 91.6285, step = 349601 (0.796 sec)
INFO:tensorflow:global_step/sec: 200.894
INFO:tensorflow:loss = 74.099, step = 349701 (0.498 sec)
INFO:tensorflow:global_step/sec: 130.776
INFO:tensorflow:loss = 50.2027, step = 349801 (0.765 sec)
INFO:tensorflow:global_step/sec: 127.64
INFO:tensorflow:loss = 38.8205, step = 349901 (0.784 sec)
INFO:tensorflow:global_step/sec: 147.723
INFO:tensorflow:loss = 42.6025, step = 350001 (0.677 sec)
INFO:tensorflow:global_step/sec: 223.095
INFO:tensorflow:loss = 48.3482, step = 350101 (0.448 sec)
INFO:tensorflow:global_step/sec: 239.987
INFO:tensorflow:loss = 63.531, step = 350201 (0.417 sec)
INFO:tensorflow

INFO:tensorflow:global_step/sec: 240.829
INFO:tensorflow:loss = 133.411, step = 357601 (0.415 sec)
INFO:tensorflow:global_step/sec: 241.548
INFO:tensorflow:loss = 31.8057, step = 357701 (0.414 sec)
INFO:tensorflow:global_step/sec: 133.296
INFO:tensorflow:loss = 63.9072, step = 357801 (0.750 sec)
INFO:tensorflow:global_step/sec: 109.607
INFO:tensorflow:loss = 105.263, step = 357901 (0.913 sec)
INFO:tensorflow:global_step/sec: 237.059
INFO:tensorflow:loss = 73.2537, step = 358001 (0.422 sec)
INFO:tensorflow:global_step/sec: 145.451
INFO:tensorflow:loss = 44.0793, step = 358101 (0.687 sec)
INFO:tensorflow:global_step/sec: 241.981
INFO:tensorflow:loss = 69.9668, step = 358201 (0.413 sec)
INFO:tensorflow:global_step/sec: 239.735
INFO:tensorflow:loss = 39.6455, step = 358301 (0.417 sec)
INFO:tensorflow:global_step/sec: 209.049
INFO:tensorflow:loss = 74.4102, step = 358401 (0.480 sec)
INFO:tensorflow:global_step/sec: 151.696
INFO:tensorflow:loss = 132.361, step = 358501 (0.659 sec)
INFO:tenso

INFO:tensorflow:global_step/sec: 173.911
INFO:tensorflow:loss = 119.033, step = 365901 (0.575 sec)
INFO:tensorflow:global_step/sec: 171.118
INFO:tensorflow:loss = 66.3425, step = 366001 (0.584 sec)
INFO:tensorflow:global_step/sec: 178.826
INFO:tensorflow:loss = 106.908, step = 366101 (0.560 sec)
INFO:tensorflow:global_step/sec: 236.198
INFO:tensorflow:loss = 45.373, step = 366201 (0.423 sec)
INFO:tensorflow:global_step/sec: 155.298
INFO:tensorflow:loss = 46.82, step = 366301 (0.644 sec)
INFO:tensorflow:global_step/sec: 171.057
INFO:tensorflow:loss = 76.5843, step = 366401 (0.585 sec)
INFO:tensorflow:global_step/sec: 236.001
INFO:tensorflow:loss = 99.537, step = 366501 (0.424 sec)
INFO:tensorflow:global_step/sec: 146.933
INFO:tensorflow:loss = 82.6842, step = 366601 (0.681 sec)
INFO:tensorflow:global_step/sec: 139.901
INFO:tensorflow:loss = 68.1073, step = 366701 (0.715 sec)
INFO:tensorflow:global_step/sec: 141.629
INFO:tensorflow:loss = 106.023, step = 366801 (0.706 sec)
INFO:tensorflo

SystemExit: 

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                        skip_blank_lines=True, keep_default_na=False)
predictions = list(temp.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
  x=df, num_epochs=1,shuffle=False)))
res = pd.DataFrame(predictions)

AttributeError: 'NoneType' object has no attribute 'predict'

[{'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.58246171], dtype=float32),
  'logits': array([ 0.33288729], dtype=float32),
  'probabilities': array([ 0.41753826,  0.58246171], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.41197711], dtype=float32),
  'logits': array([-0.35579801], dtype=float32),
  'probabilities': array([ 0.58802289,  0.41197711], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.42618015], dtype=float32),
  'logits': array([-0.29745328], dtype=float32),
  'probabilities': array([ 0.57381982,  0.42618012], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.31419533], dtype=float32),
  'logits': array([-0.7805779], dtype=float32),
  'probabilities': array([ 0.68580467,  0.31419536], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array

In [ ]:
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

'''model.train(
    input_fn=lambda: wide_deep.input_fn(
        TEST_CSV, num_epochs=100, shuffle=True, batch_size=3))'''
predictions = model.predict(
    input_fn=lambda: wide_deep.input_fn(
        TEST_CSV, num_epochs=1, shuffle=False))

#predictions = list(model.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))